This notebook is for taking our transformer models and applying SHAP explanations to their output to see which tokens contribute to what.

In [1]:
from classifiers.basic_transformer_classifier import BasicTransformerClassifier
from torch.utils.data import DataLoader
from dataset.phishing_dataset import PhishingDataset

from classifiers.basic_url_skinny_bert_classifier import BasicUrlSkinnyBertClassifier
import torch
import torch.nn as nn
from transformers import DistilBertModel, DistilBertTokenizer
import shap
import scipy as sp
import numpy as np
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

model = BasicTransformerClassifier()
checkpoint_path = "../models/canonical/basic_html_transformer_phishing_classifier.pt"
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.to(device)
model.eval()

dataset_path = os.path.expanduser("~/transfer/phishing_output_tokenized.h5")
test_dataset = PhishingDataset(['html_content'], split='dev', local_file_path=dataset_path)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

# Define a wrapper for SHAP compatibility
class BertClassifierWrapper(nn.Module):
    def __init__(self, model):
        super(BertClassifierWrapper, self).__init__()
        self.model = model

    def forward(self, tokenized_inputs):
        input_ids = tokenized_inputs.to(device)
        attention_mask = (input_ids != 0).long().to(device)
        return self.model(input_ids, attention_mask)

wrapped_model = BertClassifierWrapper(model)

feature_names = tokenizer.convert_ids_to_tokens(range(len(tokenizer)))

# prediction function
def f(x):
    tv = torch.tensor([tokenizer.encode(v, max_length=512, padding='max_length', truncation=True) for v in x])
    class_preds = wrapped_model(tv)
    outputs = class_preds.detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:, 1])
    return val

batch = test_dataloader.__iter__().__next__()
explainer = shap.Explainer(f, tokenizer)
html_content = [content[:4000] for content in batch['html_content']]
shap_values = explainer(html_content, fixed_context=1)



/opt/homebrew/Caskroom/miniforge/base/envs/phishing-edge/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/var/folders/j1/dctzbvps2w7cgcp3cflmcxm00000gn/T/ipykernel_86915/1057150206.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are ex

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer:   1%|          | 1/128 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer:   2%|▏         | 3/128 [00:57<28:55, 13.88s/it]

...

PartitionExplainer explainer:  99%|█████████▉| 127/128 [57:18<00:26, 26.09s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 100%|██████████| 128/128 [57:42<00:00, 25.65s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 129it [58:12, 27.28s/it]                         


In [17]:
file = open('temp.html','w')
file.write(shap.plots.text(shap_values[34], display=False))
file.close
#shap.plots.text(shap_values[34])

<function TextIOWrapper.close()>